### Experiments 1: portfolio optimization

* Fix set of experiments
    * stocks 
    * time interval
    * risk aversion
    * budget
* Compute exact solution for quantum formulation. Stored attributes:
    * Exact spectrum
    * Optimal bitstring
    * Optimal portfolio
    * First excited state
    * Leftover budget
    * Ratio of left over budget
    * First excited portfolio
    * Leftover budget for the excited portfolio
    * Optimal expected return, i.e., value from the objective function
    * Excited expected return
    * n_layers
    * multiple runs
* Sample expectation values from the circuit -> the experiments together show that the problem suffers from barren plateau
* Compute the classical solution with scipy and optimize allocation with LP
    * Optimal portfolio
    * Leftover budget
    * Ratio of left over budget
    * Optimal expected return

How does risk aversion affect the results?
How much does different combinations of hyperparameters, optimizers, scaler?

In [ ]:
import json
import numpy as np
import yfinance as yf
from coskweness_cokurtosis import coskewness, cokurtosis
from portfolio_hubo_qaoa import HigherOrderPortfolioQAOA

experiments = None
with open("experiments_data.json", "r") as f:
    experiments = json.load(f)

for experiment_id in experiments:
    for experiment in experiments[experiment_id]:
        print("Solving experiment")
        print(experiment)

        # Budget in dollars
        # We compute the maximum number of shares we can buy for each stock
        # At most n_qubits for each stock
        n_qubits = 4
        cheapest_stock = prices_now.min()
        budget = n_qubits*cheapest_stock
        print(budget)
        risk_aversion = 3

        data = yf.download(stocks, start="2021-01-01", end="2024-01-01")
        prices_now = data["Close"].iloc[-1]
        print(prices_now)
        returns = data["Close"].pct_change().dropna()
        stocks = returns.columns

        numpy_returns = returns.to_numpy()
        expected_returns = numpy_returns.mean(axis=0)*252
        print(returns.mean())
        print(expected_returns)
        covariance_matrix = np.cov(numpy_returns, rowvar=False)*252
        coskewness_tensor = coskewness(numpy_returns)#*(252**2)
        cokurtosis_tensor = cokurtosis(numpy_returns)#*(252**3)

        portfolio_hubo = HigherOrderPortfolioQAOA(stocks=stocks,
                                                prices_now=prices_now,
                                                expected_returns=expected_returns, 
                                                covariance_matrix=covariance_matrix,
                                                budget=budget,
                                                coskewness_tensor=coskewness_tensor, 
                                                cokurtosis_tensor=cokurtosis_tensor,
                                                log_encoding = True, 
                                                layers = 1,
                                                risk_aversion = risk_aversion)

        smallest_eigenvalues, smallest_bitstrings, first_excited_energy, optimized_portfolio, second_optimized_portfolio = portfolio_hubo.solve_exactly()